Problem 1 part 1

In [34]:
import numpy as np
x1 = np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
x2 = np.array([1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0])
p = np.array([28.1,34.3,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
m = np.matmul(p.transpose(),x1)/(np.matmul(x1.transpose(),x1))
print(m)



44.24415584415584


problem 1 part 2

In [36]:
import torch as t
from torch.autograd import Variable
